In [7]:
import sys, os
import numpy as np
import faiss
import torch
from ai import *
from langchain.agents import create_tool_calling_agent, initialize_agent, AgentExecutor, AgentType
from langchain.agents.format_scratchpad.tools import format_to_tool_messages
from langchain.agents.output_parsers.tools import ToolsAgentOutputParser
from langchain.tools import Tool
from langchain_core.agents import AgentAction, AgentFinish, AgentStep
from langchain_core.runnables import Runnable, RunnablePassthrough
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama import ChatOllama
from langgraph.prebuilt import create_react_agent
from IPython.display import Image, display
from sentence_transformers import CrossEncoder
from transformers import AutoTokenizer, AutoModel, pipeline
from tqdm import tqdm

In [2]:
# Автоперезагрузка файлов
%load_ext autoreload
%autoreload 2

In [3]:
os.environ["PYTHONDONTWRITEBYTECODE"] = "1"

Load AI

In [4]:
ai = AI()
ai.database_path = os.path.abspath("../../var/database")
ai.device = "cpu"
ai.load_store()
ai.init_llm()

In [92]:
# Определим функции
@tool
def add_tool(a, b):
    """Складывает два числа 'a' и 'b'"""
    return a + b

@tool
def multiply_tool(a, b):
    """Перемножает два числа 'a' и 'b'"""
    return a * b

@tool
def magic_function(input):
    """Applies a magic function to an input."""
    return input + 2

def find_tool(item):
    for tool in tools:
        if tool.name == item.tool:
            return tool

def next_step(items):
    
    response = []
    for item in items:
        tool = find_tool(item)
        observation = None
        
        if tool:
            observation = tool.run(item.tool_input)
        
        response.append((item, observation))
    
    return response

tools = [add_tool, multiply_tool, magic_function]
tool_calling_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Ты консультант для программистов. Отвечай на русском языке в деловом стиле"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)
first_prompt = RunnablePassthrough.assign(
    agent_scratchpad=lambda item: format_to_tool_messages(item["intermediate_steps"])
)
llm_with_tools = ai.llm_answer.bind_tools(tools)
output_parser = ToolsAgentOutputParser()
agent = create_tool_calling_agent(ai.llm_answer, tools, prompt=tool_calling_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [65]:
# Query
query = "Выполни magic_function(7)"
prompt = {"input": query}
print(agent_executor.invoke(prompt))

{'input': 'Выполни magic_function(7)? Ответ:', 'output': 'Привет! Я выполнил функцию magic_function с аргументом 7 и получил результат: 9. Это может быть каким-то магическим эффектом, который я не могу описать подробнее.'}


In [ ]:
# Query
query = "Сколько будет 5 + 7. Ответ:"

# Get tool
prompt = {"input": query, "intermediate_steps": []}
prompt = first_prompt.invoke(prompt)
prompt = tool_calling_prompt.invoke(prompt)
prompt = llm_with_tools.invoke(prompt)
response = output_parser.invoke(prompt)
print(response)

# Answer
prompt = {"input": query, "intermediate_steps": next_step(response)}
prompt = first_prompt.invoke(prompt)
prompt = tool_calling_prompt.invoke(prompt)
for chunk in ai.llm_answer.stream(prompt, stream=True):
    print(chunk.content, end="", flush=True)

#prompt = llm_with_tools.invoke(prompt)
#answer = output_parser.invoke(prompt)
#print(answer)

In [ ]:
# Get tool
prompt = {"input": query, "intermediate_steps": []}
prompt = first_prompt.invoke(prompt)
prompt = tool_calling_prompt.invoke(prompt)
for chunk in ai.llm_answer.stream(prompt, stream=True):
    print(chunk.content, end="", flush=True)

#prompt = ai.llm_answer.invoke(prompt)
#response = output_parser.invoke(prompt)
#print(response)

In [72]:
# Answer
prompt = {"input": query, "intermediate_steps": next_step(response)}
print(agent.invoke(prompt))

return_values={'output': 'Привет! Я выполнил функцию magic_function с аргументом 7 и получил результат: 0. Это может быть каким-то умным образом, но в данном случае оно не указывает на конкретный результат. Если ты хочешь узнать больше деталей, я могу попробовать объяснить это через другие функции или методы.'} log='Привет! Я выполнил функцию magic_function с аргументом 7 и получил результат: 0. Это может быть каким-то умным образом, но в данном случае оно не указывает на конкретный результат. Если ты хочешь узнать больше деталей, я могу попробовать объяснить это через другие функции или методы.'
